## Načítání dat ze souboru

## čistý python

In [ ]:
f = open("./data/pyvenv.cfg", "r") # r -read, w - write, rw, rb - read binary, wb write binary

# data = f.read() #  - nacte cely obsah souboru

line = f.readline()
line = f.readline()
print(line)
lines = f.readlines()
print(lines)


if not f.closed:
    f.close()

In [ ]:
f = open("./data/pyvenv.cfg", "r") # r -read, w - write, rw, rb - read binary, wb write binary
print(f.tell()) # tell() nam rika, na jake pozici v souboru se nachazime

line = f.readline()
print(f.tell())
print(line)
f.seek(0, 0) # f.seek(count, whence), whence = 0, 1, 2
line = f.readline()
print(line)
print(f.tell())

if not f.closed:
    f.close()

In [ ]:
f = open("./data/pyvenv.cfg", "rb") # r -read, w - write, rw, rb - read binary, wb write binary

f.seek(-16,2)

line = f.readline()

print(line.decode('utf-8'))

if not f.closed:
    f.close()

In [ ]:
settings = {}
with open("./data/pyvenv.cfg", "r") as f: # with = neco jako try: a except:
    line = f.readline()
    while line:
        line = line.strip()
        key, val = line.split(" = ")
        settings[key] = val
        line = f.readline()

print(settings)

print(f.closed)

## csv

- comma separated values

In [ ]:
with open("./data/topeni.txt", "r") as f:
    for _ in range(5):
        print(f.readline())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("./data/topeni.txt").transpose()
print(data)
plt.plot(data[1][:12], data[2][:12])
plt.show()

In [ ]:
# csv reader
import csv

with open("./data/topeni.txt") as csvfile:
    reader = csv.reader(csvfile, delimiter = ' ')

    for row in reader:
        print(row)

In [ ]:
import pandas as pd

file = "./data/topeni.txt"

df = pd.read_csv(file, sep = " ", skiprows = 1, names = ["year", "month", "heat", "price"])
df.head()

## JSON


In [ ]:
import json

with open("./data/example.json", "r") as f:
    data = f.read()

d = json.loads(data)
for key, val in d["glossary"].items():
    print(key, val)
    
def pretty_print_dict(d, indent=0): # hezke cviceni, rekurzivni vypis slovniku
    pass

print(json.dumps(d, indent = 2))

In [ ]:
import json

with open("./data/bibliography.json", "r") as f:
    d = json.load(f)

print(d[0]["DOI"])

## XML
- eXtensible Markup Language

In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse('./data/sample.xml')
root = tree.getroot()

for child in root:
    print(child.attrib["name"])
    for grandchild in child:
        print("\t",grandchild.text)
    print("\tSousede:")
    for neighbor in child.iter("neighbor"):
        print("\t\t", neighbor.attrib["name"])

## .ini

- Windows konfiguracni soubor

```
key = value
```

In [ ]:
import configparser as cp

config = cp.ConfigParser()
config.read("./data/sample.ini")

print(config["mysql"]["user"])

for section in config:
    print(section)
    for entry in config[section]:
        print("\t", entry, config[section][entry])

In [ ]:
lst = config["postgresql"]["lst"]
print(type(lst))
x = eval(lst)
print(type(x))

## Excel

xlrd

```
pip3 install xlrd
```

## API

- Application Programming Interface
- zde typicky:
    - pres HTTP - hyperttext transfer protocol
    - dostaneme text, JSON

In [ ]:
# balik requests: pip3 install requests
# obstarava HTTP komunikace

In [ ]:
import requests

url = "https://vaclav-alt.github.io/data"

r = requests.get(url)
print(r.status_code) # 200 - OK, 404 - not found

if r.status_code == 200:
    print(r.text)

In [ ]:
import requests
import json
from IPython.display import Image, display

url = "https://dog.ceo/api/breeds/image/random"

r = requests.get(url)
if r.status_code == 200:b
    rawjson = r.text
    data = json.loads(rawjson)
    display(Image(data["message"]))

In [ ]:
import requests
import json

url = "https://v2.jokeapi.dev/joke/Any?blacklistFlags=nsfw&amount=2"

r = requests.get(url)
if r.status_code == 200:
    data = json.loads(r.text)
    print(json.dumps(data, indent = 2))

## Zabezpecena api

In [ ]:
import requests

# api token / authentization token / athorization token
api_token = "br38fhf87q8rc9028wacnfjq8247rb8fw"  # typicky vypada nejak takhle

url = "https://www.github.com/vaclav-alt/repos&parametr1=4"
r = request.get(url, headers = {'Authorization' : api_token})

# dal uplne stejne

## Webscraping a beautiful soup

beuatiful soup 4 - balik na parsovani HTML
```
pip3 install bs4 html5lib lxml
```

In [ ]:
from bs4 import BeautifulSoup

with open("data/sample.html", "r") as f:
    soup = BeautifulSoup(f, "html.parser")
#print(soup.table)
#print(soup.table.attrs)

tbl = soup.find(id = "sample_table")
rows = tbl.find_all("tr")
tds = rows[0].find_all("td")
print(tds[1].text)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://vaclav-alt.github.io/data"
r = requests.get(url)

if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")

dates = []
temps = []

tbls = soup.find_all("table")

for tbl in tbls:
    day = tbl.caption.text
    rows = tbl.find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        time = cols[0].text
        temp = cols[1].text
        date = day + " " + time
        dates.append(date)
        temps.append(temp)

df = pd.DataFrame({"temp" : temps})
df = df.astype({"temp" : float})
df.index = pd.to_datetime(dates)

df2 = df.resample("2H").mean()
df2.plot()